# Step 1: Get the Git Repo

This is forked repo from the darknet.
Modifications are made for this project.

In [ ]:
! git clone https://github.com/alekhyaramarao/darknet.git
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

#Step 2:
For the purpose of reading data use google drive.
*As adding data in the colab runtime will be repeated and cost ineffective.*


In [ ]:
# connect to the google drive
from google.colab import drive
# drive.mount('/content.gdrive')
drive._mount('/content/drive')


Mounted at /content/drive



## Step 3: Data Preprocessing

# ***Skip to Step 7 if not training***

The below code will execute the following files:

1. **bosch_to_pascal.py** - Converts the bosch data to VOC format and stores it in the given path.
2.   **make_xml_list.py** - Read all the xml files from the given path and create "bosch_traffic_light_xmls_list.txt"
3.   **bosch_voc_to_yolo_converter.py** - Reads the bosch_traffic_lights_xmls_list.txt and converts the bosch VOC data to YOLO type by creating individual .txt file for every image in the given path. *( For this framework, ensure that the path given to store the .txt files is the same where the images are store)*
4. **train_test_split.py** - This will create a split of the given percentage between train and validation data.
(If you are running for just generating a result file for the input to classification training, then no need to split)







In [ ]:
% cd traffic-lights/
! python bosch_to_pascal.py train.yaml /content/drive/MyDrive/Dataset/rgb/train/traffic_light_xmls/
! python make_xml_list.py /content/drive/MyDrive/Dataset/rgb/train/traffic_light_xmls/
! python bosch_voc_to_yolo_converter.py /content/drive/MyDrive/Dataset/rgb/train/traffic_light_images/ bosch_traffic_light_xmls_list.txt /content/drive/MyDrive/Dataset/rgb/train/traffic_light_images/
! python train_test_split.py traffic_lights.txt 0.2

%cd ..

# Step 4: Get Weights
(Only required if training and for the first time)
If continuing training then use the previously found best model weights and start training from there


In [ ]:
# get yolov4 weights
#only run this if starting the training from the scratch. If you have already trained and want to train from where you left off, then use those weights. 
! wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29


# Step 5: Install Pre-requisites

In [ ]:
#install all the apps which are required
!apt install ffmpeg libopencv-dev libgtk-3-dev python-numpy python3-numpy libdc1394-22 libdc1394-22-dev libjpeg-dev libtiff5-dev libavcodec-dev libavformat-dev libswscale-dev libxine2-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libv4l-dev libtbb-dev qtbase5-dev libfaac-dev libmp3lame-dev libopencore-amrnb-dev libopencore-amrwb-dev libtheora-dev libvorbis-dev libxvidcore-dev x264 v4l-utils unzip

# Step 6: Start Training

In [ ]:
# %cd ..
%cd /content/darknet/
! ./darknet detector train traffic-lights/voc-bosch.data traffic-lights/yolov4-tiny-custom.cfg traffic-lights/yolov4-tiny-custom_best.weights -dont_show -map

# Step 7: Test
**Skip steps 3-6 if only testing the model**
Once trained use the best weights and input video/Image to test your trained model

In [ ]:
# ! ./darknet detector demo voc-bosch.data yolov4-tiny-custom.cfg yolov4-tiny-custom_best.weights test_video.mp4 -dont_show -out_filename res.mp4


In [ ]:
# ! ./darknet detector test voc-bosch.data yolov4-tiny-custom.cfg yolov4-tiny-custom_best.weights 39418.png -dont_show

# Step 8: Generate Json file with predicted bounding box information using trained weights

Pass a .txt file with the path to images

***This is required as entry point for classification network***

Follow Step 3 to create .txt file with all images from the dataset.

In [ ]:
! ./darknet detector test traffic-lights/voc-bosch.data traffic-lights/yolov4-tiny-custom.cfg traffic-lights/yolov4-tiny-custom_best.weights -ext_output -dont_show -out result.json < traffic-lights/traffic_lights.txt